In [34]:
import numpy as np
import pandas as pd
import datasets

import gadgets

In [35]:
orig_aqua = datasets.load_dataset("aqua_rat")
orig_aqua_tokenized = datasets.load_dataset("aqua_rat", "tokenized")

Generating train split:   0%|          | 0/97467 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/254 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/254 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/97467 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/254 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/254 [00:00<?, ? examples/s]

In [36]:
orig_aqua["train"][0]

{'question': "Two friends plan to walk along a 43-km trail, starting at opposite ends of the trail at the same time. If Friend P's rate is 15% faster than Friend Q's, how many kilometers will Friend P have walked when they pass each other?",
 'options': ['A)21', 'B)21.5', 'C)22', 'D)22.5', 'E)23'],
 'rationale': 'If Q complete x kilometers, then P completes 1.15x kilometers.\nx + 1.15x = 43\n2.15x=43\nx = 43/2.15 = 20\nThen P will have have walked 1.15*20=23 km.\nThe answer is E.',
 'correct': 'E'}

In [37]:
import random
import string

id_chars = list(string.ascii_letters + string.digits)

aqua = {}

def parse_options(options: list[str]) -> dict[str, str]:
    assert len(options) == 5
    options_dict = {"A": None, "B": None, "C": None, "D": None, "E": None}
    for option, letter in zip(options, options_dict.keys()):
        prefix = letter + ")"
        assert option.startswith(prefix)
        options_dict[letter] = option.removeprefix(prefix).strip()
    return options_dict

for split in orig_aqua.keys():
    random.seed(f"aqua_rat/{split}")
    aqua[split] = orig_aqua[split].to_pandas()
    aqua[split]["id"] = "aqua_rat__" + pd.Series(["".join(random.choices(id_chars, k=16)) for _ in range(len(aqua[split]))])
    aqua[split]["options"] = aqua[split]["options"].apply(parse_options)
    aqua[split]["correct"] = aqua[split]["correct"].str.upper().str.strip()
    aqua[split]["tokenized_question"] = orig_aqua_tokenized[split]["question"]
    assert set(aqua[split]["correct"].unique()) == {"A", "B", "C", "D", "E"}


In [38]:
import pandarallel

pandarallel.pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 40 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [39]:
def transform_chain(row) -> str:
    example = gadgets.aqua.parse(row.to_dict())
    markup = gadgets.markup.to_model_markup(example=example)
    return str(markup)

for split in orig_aqua.keys():
    aqua[split]["chain"] = orig_aqua[split].to_pandas().parallel_apply(transform_chain, axis=1)

/home/xkadlci2/gadgets/gadgets/markup.py:14: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  return bs4.BeautifulSoup(step, features="html.parser")
/home/xkadlci2/gadgets/gadgets/markup.py:14: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  return bs4.BeautifulSoup(step, features="html.parser")
/home/xkadlci2/gadgets/gadgets/markup.py:14: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  return bs4.BeautifulSoup(step, features="html.parser")
/home/xkadlci2/gadgets/gadgets/markup.py:14: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  return bs4.Beautifu

/home/xkadlci2/gadgets/gadgets/markup.py:14: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  return bs4.BeautifulSoup(step, features="html.parser")
/home/xkadlci2/gadgets/gadgets/markup.py:14: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  return bs4.BeautifulSoup(step, features="html.parser")
/home/xkadlci2/gadgets/gadgets/markup.py:14: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  return bs4.BeautifulSoup(step, features="html.parser")
/home/xkadlci2/gadgets/gadgets/markup.py:14: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  return bs4.Beautifu

/home/xkadlci2/gadgets/gadgets/markup.py:14: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  return bs4.BeautifulSoup(step, features="html.parser")
/home/xkadlci2/gadgets/gadgets/markup.py:14: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  return bs4.BeautifulSoup(step, features="html.parser")
/home/xkadlci2/gadgets/gadgets/markup.py:14: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  return bs4.BeautifulSoup(step, features="html.parser")
/home/xkadlci2/gadgets/gadgets/markup.py:14: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  return bs4.Beautifu

In [40]:
prompt_selects = ["Options", "Choices", "Choose one", "Pick one", "Select the correct option", "Select", "Pick", "Answers", "Choose the most appropriate option", "Choose the correct option", "Choose the correct answer", "Choose the correct choice", "Choose the correct choice from the following", "Choose the correct choice from the following options", "Choose the correct choice from the following answers", "Choose the correct choice from the following answers", "Choose the correct choice from the following choices"]
prompt_select_seps = [".\n", ":", ":\n", "\n"]
whitespace_seps = [" ", "  ", "\n", "\t"]

def format_question_with_options(question: str, options: dict[str, str]):
    prompt_select = random.choice(prompt_selects)
    sep_1 = random.choice(whitespace_seps)
    sep_2 = random.choice(prompt_select_seps)
    sep_3 = random.choice(whitespace_seps)
    if not sep_2.endswith("\n"):
        sep_2 += sep_3
    return question + sep_1 + prompt_select + sep_2 + sep_3.join(f"{k.upper()}) {v}" for k, v in options.items())
    

In [41]:
aqua["train"].head()
random.seed(42)
for split in aqua.keys():
    aqua[split] = pd.DataFrame({
        "id": aqua[split]["id"],
        "question": [format_question_with_options(q, o) for q, o in zip(aqua[split]["question"], aqua[split]["options"])],
        "chain": aqua[split]["chain"],
        "result": aqua[split]["correct"],
        "options": aqua[split]["options"],
        "question_without_options": aqua[split]["question"],
    })
    

In [42]:
aqua["train"].iloc[0].to_dict()

{'id': 'aqua_rat__TeUuCx2QFEzwmZzg',
 'question': "Two friends plan to walk along a 43-km trail, starting at opposite ends of the trail at the same time. If Friend P's rate is 15% faster than Friend Q's, how many kilometers will Friend P have walked when they pass each other? Pick one:\nA) 21  B) 21.5  C) 22  D) 22.5  E) 23",
 'chain': 'If Q complete x kilometers, then P completes 1.15x kilometers.   x + 1.15x = 43   2.15x=43   x = 43/2.15= \n<gadget id="calculator">43/2.15</gadget>\n<output>20</output>\n20   Then P will have have walked   1.15*20= \n<gadget id="calculator">  1.15*20</gadget>\n<output>23</output>\n23 km.   The answer is 23.\n<result>E</result>',
 'result': 'E',
 'options': {'A': '21', 'B': '21.5', 'C': '22', 'D': '22.5', 'E': '23'},
 'question_without_options': "Two friends plan to walk along a 43-km trail, starting at opposite ends of the trail at the same time. If Friend P's rate is 15% faster than Friend Q's, how many kilometers will Friend P have walked when they p

In [43]:
ds = datasets.DatasetDict({
    "train": datasets.Dataset.from_pandas(aqua["train"]),
    "validation": datasets.Dataset.from_pandas(aqua["validation"]),
    "test": datasets.Dataset.from_pandas(aqua["test"]),
})

#ds.push_to_hub("MU-NLPC/Calc-aqua_rat", config_name="original-splits")

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/98 [00:00<?, ?ba/s]

Deleting unused files from dataset repository:   0%|          | 0/1 [00:00<?, ?it/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Deleting unused files from dataset repository:   0%|          | 0/1 [00:00<?, ?it/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Deleting unused files from dataset repository:   0%|          | 0/1 [00:00<?, ?it/s]